# Modelling

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, f1_score, roc_curve, auc
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import shap
import joblib



# Preparing our dataset

In [ ]:
# Load our data (cleaning and outlier techniques applied)
df = pd.read_csv('churn_cleaned_df.csv')
df.head()

In [ ]:
# Drop the column and save it to a variable
#   If you need to bring the column back later, simply reassign it to the DataFrame
#   df['customerID'] = customer_id
customer_id = df.pop('customerID')

# Separate the target variable (Churn) from the rest of the dataset
X = df.drop('Churn', axis=1)
y = df['Churn']

# Balancing our training dataset
Given that our dataset is imbalanced (26.5% of the users churn), we need to correct this effect in order to obtain the desired results with the model.

SMOTE (Synthetic Minority Over-sampling Technique) is a popular algorithm used for data upsampling in machine learning. It creates synthetic samples of the minority class by creating new observations that are similar to existing observations, thus balancing the class distribution. We can perform data upsampling using SMOTE in Python with the imbalanced-learn library.

Balancing your dataset with SMOTE should be performed on the training set only. This way, you ensure that your model learns from a balanced representation of the data while preserving the integrity and independence of the test and validation datasets. In other words, you avoid information leakage, realistic representation of the data and an independent evaluation of the model.

In [ ]:
# Instantiate the SMOTE object
oversample = SMOTE()

# Apply SMOTE to upsample the minority class (Churn = 1) on the training set only
X_resampled_train, y_resampled_train = oversample.fit_resample(X, y)

# Combine the resampled features and target variable into a new training DataFrame
resampled_df_train = pd.concat([X_resampled_train, y_resampled_train], axis=1)

# Calculate the churn and non-churn counts
churn_count = resampled_df_train[resampled_df_train['Churn'] == 1].shape[0]
non_churn_count = resampled_df_train[resampled_df_train['Churn'] == 0].shape[0]

# Set up the plot size and title
plt.figure(figsize=(8, 6))
plt.title('Churn Rate')

# Set the colors for the pie chart sections
colors = ['lightcoral', 'lightgreen']

# Create the pie chart
plt.pie([churn_count, non_churn_count], labels=['Churn', 'Non-Churn'], autopct='%1.1f%%', colors=colors)

# Display the plot
plt.show()

# Split the dataset
Divide the dataset into training, validation, and test sets. The typical split is 70-80% for training, 10-15% for validation, and 10-15% for testing. The validation set helps in model selection and hyperparameter tuning.

Therefore, in order to obtain a split of 70% for training, 15% for validation, and 15% for testing we are setting the test dataset with a size of 15% of the original dataset, while the combined training/validation set is split with a test size of 0.1765 (approximately 15% of the combined training/validation data).

In [ ]:
# Re-assign to variables 
X, y = X_resampled_train, y_resampled_train

In [ ]:
# Split the data into training, validation, and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42, stratify=y_train_val)


# Model selection
We are exploring different options for our churn prediction, from more simple and interpretable models (i.e. Logistic Regression) to advanced models (i.e. SVM or XGBoost). We will decide which one to pick based on the validation set's performance metrics.

In [ ]:
# Define a list of models including additional choices
models = [
    LogisticRegression(max_iter=1000),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    SVC(probability=True),
    XGBClassifier(),
    MLPClassifier(max_iter=1000)
]

# Iterate over the models
for model in models:
    # Train the model on the resampled training set
    model.fit(X_train, y_train)

    # Make predictions on the validation set
    y_pred_val = model.predict(X_val)

    # Calculate prediction probabilities on the validation set
    y_prob_val = model.predict_proba(X_val)[:, 1] # Probability of the positive class (churn)

    # Evaluate the model performance on the validation set
    accuracy_val = accuracy_score(y_val, y_pred_val)
    recall_val = recall_score(y_val, y_pred_val)
    precision_val = precision_score(y_val, y_pred_val)
    f1_val = f1_score(y_val, y_pred_val)
    cm_val = confusion_matrix(y_val, y_pred_val)
    tn_val, fp_val, fn_val, tp_val = cm_val.ravel()
    specificity_val = tn_val / (tn_val + fp_val)

    # Print evaluation metrics on the validation set for each model
    print(f"Model: {type(model).__name__}")
    print("Model classification report")
    print(f"Accuracy: {accuracy_val:.4f}")
    print(f"Recall: {recall_val:.4f}")
    print(f"Precision: {precision_val:.4f}")
    print(f"F1 Score: {f1_val:.4f}")
    print(f"Confusion Matrix: \n{cm_val}")
    print(f"Specificity: {specificity_val:.4f}")
    print("\n")

XGBoost (Extreme Gradient Boosting) shows a slightly higher F1 score compared to other models like Logistic Regression. It is often a popular choice for many data science projects because it offers high performance and efficiency with parallel processing and tree pruning. Moreover it provides built-in regularization and handles missing values, making it versatile for regression and classification tasks. XGBoost's ensemble method yields accurate predictions, and its feature importance analysis aids in understanding data patterns

# Hyperparameter tuning
Properly tuning these hyperparameters can significantly improve the model's accuracy, prevent overfitting, and ensure optimal performance on unseen data. Without effective hyperparameter tuning, the model may underperform, leading to suboptimal results and wasted computational resources.

In [ ]:
# Create an XGBoost classifier object
xgb_clf = XGBClassifier()

# Define the hyperparameter grid for GridSearchCV
param_grid = {
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 300]
}

# Create the GridSearchCV object with the classifier, hyperparameter grid, and evaluation metric
grid_search = GridSearchCV(xgb_clf, param_grid, scoring='f1', cv=5)

# Fit the GridSearchCV on the validation data
grid_search.fit(X_val, y_val)

# Get the best hyperparameters found by GridSearchCV
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Evaluating our tuned model

In [ ]:
# Create a new XGBoost classifier with the best hyperparameters
best_xgb_clf = XGBClassifier(**best_params)

# Train the classifier on the training set
best_xgb_clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred_test = best_xgb_clf.predict(X_test)

# Evaluate the model performance on the test set
accuracy_test = accuracy_score(y_test, y_pred_test)
recall_test = recall_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test)
f1_test = f1_score(y_test, y_pred_test)
cm_test = confusion_matrix(y_test, y_pred_test)
tn_test, fp_test, fn_test, tp_test = cm_test.ravel()
specificity_test = tn_test / (tn_test + fp_test)

# Print evaluation metrics on the test set
print("\nTest classification report")
print(f"Accuracy: {accuracy_test:.4f}")
print(f"Recall: {recall_test:.4f}")
print(f"Precision: {precision_test:.4f}")
print(f"F1 Score: {f1_test:.4f}")
print(f"Confusion Matrix: \n{cm_test}")
print(f"Specificity: {specificity_test:.4f}")

After tuning our XGBoost model we can determine that:
- Our model achieved an accuracy of 85.25% on the test data, meaning that 85.25% of the customers were correctly classified as churn or non-churn users.

- The recall, which measures our ability to identify churn users accurately, was at an impressive 88.02%. This indicates that we correctly identified 88.02% of the customers who were likely to churn, which is crucial for focusing our retention efforts on those at risk.

- The precision, representing our accuracy in predicting churn correctly, stood at 83.39%. This means that when we predict a customer will churn, we can be confident that around 83.39% of the time, our prediction is accurate.

- The F1 score, which balances precision and recall, was at 85.64%. This shows that our model has a good balance between accurately predicting churn and identifying most of the actual churners. 

- However specificity decreased compared to our initial model, showing that our hypertuned model shows a worse performance identifying non-churn users.

In [ ]:
# Create dataframe with test labels and our predictions
final_df = pd.DataFrame({'prediction_churn': y_pred_test, 'is_churn': y_test})

# Calculate percentage of churned users we predicted to be churn
#  We arrive at the same figure as Recall
final_df[(final_df['is_churn']==1) & (final_df['prediction_churn']==1)].shape[0] / final_df[final_df['is_churn']==1].shape[0]

# Model interpretability
Model interpretability is important for building trust between stakeholders, understanding model behavior, identifying and resolving errors, ensuring regulatory compliance, facilitating collaboration, and guiding model improvement. It plays a critical role in the practical application of machine learning models and enables their effective and responsible use in real-world scenarios.

In [ ]:
# Create an explainer object by providing our XGBoost model,
explainer = shap.Explainer(best_xgb_clf)

# Calculate SHAP value using our testing set
shap_values = explainer(X_test)

# Create a summary plot
shap.summary_plot(shap_values, X_test)

### How should you interpret it?
MonthlyCharges being blue on the left and red on the right, it means that low values of MonthlyCharges have a negative impact on the predicted target value, whereas high values of MonthlyCharges have a positive impact to the output.

Similarly, with tenure being blue on the right and red on the left, it means that low values of tenure have a positive impact on the predicted target value, whereas high values of tenure have a negative impact.

In [ ]:
shap.plots.bar(shap_values, max_display=15)

Overall, these results suggest that pricing plans, contract terms and how long are customers engaged with the company (tenure) are important factors in predicting customer churn. The model may be used to optimize pricing plans and contract terms to minimize churn and maximize customer retention.

In [ ]:
# Forceplot for first observation
shap.initjs()
shap.plots.force(shap_values[38]) 

In [ ]:
# Plot the SHAP values in a different format
shap.plots.waterfall(shap_values[38])

# Storing the trained model
Storing the model locally provides convenience, reproducibility, deployment flexibility, and collaboration capabilities. It allows you to leverage the trained model for making predictions, sharing, and working on further improvements without the need for retraining or relying on the original training data.

We will be using `joblib`, a Python library that provides tools for efficient and easy serialization (saving) and deserialization (loading) of Python objects. It is primarily used for handling large data structures, such as NumPy arrays or scikit-learn models, in a way that minimizes memory usage and serialization time. It provides improved efficiency, memory management, and parallel processing capabilities compared to the standard pickle module. Another alternative would be storing the model using `pickle`

In [ ]:
# Save the trained model to a file
model_dump = 'churn_prediction_xgboost_model.joblib'
joblib.dump(model, model_dump)

In [ ]:
# Load the saved model from file
loaded_model = joblib.load('churn_prediction_xgboost_model.joblib')

# Read one row of data for demonstration purposes
X_new = pd.read_csv('churn_cleaned_df.csv').tail(1)
X_new = X_new.drop(['Churn', 'customerID'], axis=1)

# Use the loaded model for predictions
y_pred = loaded_model.predict(X_new)

# Print results
y_pred